In [39]:
import pandas as pd 
import numpy as np 

In [40]:
df = pd.read_csv("/Users/almarai/Private Projects/Pacify/data/Protocol/PAMAP2_MERGED.csv")

df.head()

,timestamp,activityID,heart_rate,hand_col_1,hand_col_2,hand_col_3,hand_col_4,hand_col_5,hand_col_6,hand_col_7,...,ankle_col_9,ankle_col_10,ankle_col_11,ankle_col_12,ankle_col_13,ankle_col_14,ankle_col_15,ankle_col_16,ankle_col_17,subjectID
0,5.89,0,NaN,33.3125,-9.79760,-1.45670,1.014380,-9.70702,-1.60337,1.32098,...,0.010364,0.010731,-35.4550,38.3829,-17.9083,0.546211,0.370903,0.695334,0.283892,subject108
1,5.90,0,NaN,33.3125,-9.91204,-1.41822,0.974050,-9.78262,-1.60285,1.29085,...,-0.005926,-0.006704,-35.4508,38.2543,-17.7663,0.545806,0.370867,0.695680,0.283872,subject108
2,5.91,0,NaN,33.3125,-9.87531,-1.57027,0.975113,-9.78237,-1.57265,1.29079,...,-0.002265,0.014646,-35.6984,38.2688,-17.0580,0.545484,0.371607,0.695437,0.284117,subject108
3,5.92,0,NaN,33.3125,-9.72175,-1.60870,1.054520,-9.70677,-1.60334,1.35118,...,0.035314,0.010982,-34.9492,38.7447,-17.6233,0.545474,0.372077,0.694859,0.284935,subject108
4,5.93,0,NaN,33.3125,-9.79920,-1.49497,0.975999,-9.58657,-1.57408,1.27549,...,-0.001065,-0.000410,-35.2105,38.8857,-17.9080,0.545810,0.372349,0.694122,0.285732,subject108


In [41]:
df = df[(df["activityID"] != 0)][["activityID", "heart_rate"]]
df.head()

,activityID,heart_rate
6147,1,84.0
6148,1,NaN
6149,1,NaN
6150,1,NaN
6151,1,NaN


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1942872 entries, 6147 to 2863455
Data columns (total 2 columns):
 #   Column      Dtype  
---  ------      -----  
 0   activityID  int64  
 1   heart_rate  float64
dtypes: float64(1), int64(1)
memory usage: 44.5 MB


In [43]:
df.dropna(inplace = True)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177408 entries, 6147 to 2863454
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   activityID  177408 non-null  int64  
 1   heart_rate  177408 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 4.1 MB


In [45]:
df.head()

,activityID,heart_rate
6147,1,84.0
6158,1,84.0
6169,1,84.0
6180,1,84.0
6191,1,84.0


Now we create a column for the bpm based on the heart rate. This value will currently be chosen in a random fashion, but the selection of this value will be more advanced in advanced versions. 

In [46]:
import random

def map_music_bpm(hr):
    if not isinstance(hr, (int, float)):
        raise ValueError("Heart rate must be a number.")
    if hr <= 90:
        return random.uniform(60, 80)
    elif hr <= 120:
        return random.uniform(100, 120)
    elif hr <= 150:
        return random.uniform(130, 150)
    else:
        return random.uniform(160, 180)




In [49]:
df["bpm"] = df["heart_rate"].apply(map_music_bpm)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177408 entries, 6147 to 2863454
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   activityID  177408 non-null  int64  
 1   heart_rate  177408 non-null  float64
 2   bpm         177408 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 5.4 MB


Note the indices error you were getting with the previous code. This is because when you create and individual series of bpm_values, the indices don't match the row indices with the heart_rate values (because you droppped null values). So we use the apply function to avoid the error of misplaced indices. Another fix would be to reset the indices of the dataset and then apply the function.  

Now we our final dataset, we train the knn regressor. 

In [51]:
from sklearn.model_selection import train_test_split 

X = df.drop(columns = ["bpm"])
y = df["bpm"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [52]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ✅ 4. KNN Regression
knn = KNeighborsRegressor(n_neighbors=5)  # k=5, you can tune this

# Fit the model
knn.fit(X_train, y_train)

# Predict
y_pred_knn = knn.predict(X_test)

# ✅ 5. Evaluate KNN Regression
mae_knn = mean_absolute_error(y_test, y_pred_knn)
rmse_knn = np.sqrt(mean_squared_error(y_test, y_pred_knn))
r2_knn = r2_score(y_test, y_pred_knn)

print("KNN Regression:")
print(f" MAE: {mae_knn:.3f}")
print(f" RMSE: {rmse_knn:.3f}")
print(f" R^2: {r2_knn:.3f}")

KNN Regression:
 MAE: 5.326
 RMSE: 6.318
 R^2: 0.962


In [53]:
# ✅ 6. Multiple Linear Regression
linreg = LinearRegression()

# Fit the model
linreg.fit(X_train, y_train)

# Predict
y_pred_lr = linreg.predict(X_test)

# ✅ 7. Evaluate Multiple Linear Regression
mae_lr = mean_absolute_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

print("\nMultiple Linear Regression:")
print(f" MAE: {mae_lr:.3f}")
print(f" RMSE: {rmse_lr:.3f}")
print(f" R^2: {r2_lr:.3f}")


Multiple Linear Regression:
 MAE: 10.245
 RMSE: 12.364
 R^2: 0.853


Now we also perform cross-validation to correctly interpret our results and see if our KNN model is overfitting the data, and we also try one tree based Regressor, namely RandomForestRegressor.

In [54]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

knn2 = KNeighborsRegressor(n_neighbors=5)
linreg2 = LinearRegression()
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# 3. Run CV
for model, name in zip([knn, linreg, rf], ["KNN", "Linear Regression", "Random Forest"]):
    scores = cross_val_score(model, X, y, cv=5, scoring='r2')
    print(f"\n{name} CV R²: {scores}")
    print(f"{name} Mean R²: {scores.mean():.4f} ± {scores.std():.4f}")


KNN CV R²: [0.96614309 0.957088   0.94528366 0.96399639 0.96091781]
KNN Mean R²: 0.9587 ± 0.0074

Linear Regression CV R²: [0.85205511 0.849101   0.80223027 0.86558403 0.82757679]
Linear Regression Mean R²: 0.8393 ± 0.0222

Random Forest CV R²: [0.97107019 0.96440795 0.95221714 0.96835068 0.96758262]
Random Forest Mean R²: 0.9647 ± 0.0066


We can see that in the case of our dataset, the RandomForestRegressor has the best R^2 Score, which usually translates to a good prediction algorithm for the ideal beats per minute. 

From analysis, we know that a linear model between the data is more practical, than a KNN model because the data has many points of bpm in a specific heart range, thus allowing the model to predict in the given heart range well. For bumpier data a linear model would be much more practical. So we follow through with a linear model which can be altered as we change our model later on. 

In [57]:
import joblib

# 2. Save the model
joblib.dump(linreg, '/Users/almarai/Private Projects/Pacify/backend/models/linear_regression_model.pkl')
print("✅ Model saved!")

✅ Model saved!


In [59]:
# 3. Load the model later
loaded_model = joblib.load('/Users/almarai/Private Projects/Pacify/backend/models/linear_regression_model.pkl')
print("✅ Model loaded!")

# 4. Predict with the loaded model
y_pred = loaded_model.predict([[1, 85]])
print("Predicted value:", y_pred)

✅ Model loaded!
Predicted value: [84.33025453]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Now we have successfully created and saved our linear regression model. 